## Cargar Dataframes


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, lit
from pyspark.sql.functions import col, count, avg, sum, max, min, when, datediff, current_date
import re

In [2]:
# ==============================
# 1. Inicializar Spark Session
# ==============================
spark = SparkSession.builder \
    .appName("HDFS_NiFi_Data_Cleaning") \
    .master("spark://spark-master:7077") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.cores", "2") \
    .config("spark.cores.max", "4") \
    .config("spark.driver.memory", "1g") \
    .config("spark.driver.host", "jupyter") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .getOrCreate()

print("Sesión creada correctamente")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/18 02:19:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/18 02:19:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/18 02:19:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Sesión creada correctamente


In [3]:
# ==============================
# 2. Paths en HDFS
# ==============================
hdfs_base_path = "hdfs://namenode:9000/user/nifi/"

# --- CSVs ---
hdfs_path_maestra = hdfs_base_path + "maestra.csv"
hdfs_path_zona = hdfs_base_path + "bd_zona.csv"
hdfs_path_json = hdfs_base_path + "datos.json"
hdfs_path_val_prod = hdfs_base_path + "Valores-Venta_Producto.csv"
hdfs_path_uni_prod = hdfs_base_path + "Unidades-Venta_Producto.csv"
hdfs_path_val_fam = hdfs_base_path + "Valores-Venta_Familia.csv"
hdfs_path_uni_fam = hdfs_base_path + "Unidades-Venta_Familia.csv"

In [4]:
# ===============================================
# 2. Carga de Datos en DataFrames de Spark
# ===============================================

print("--- 1. Cargando Archivos Maestros (Separador ';') ---")
# Primer CSV (Maestra)
print("--- CSV Maestra ---")
df_maestra = spark.read.csv(hdfs_path_maestra, header=True, inferSchema=True, sep=";")
df_maestra.show(5, truncate=False)

# Segundo CSV (Zona)
print("--- CSV Zona ---")
df_zona = spark.read.csv(hdfs_path_zona, header=True, inferSchema=True, sep=";")
df_zona.show(5, truncate=False)


print("--- 2. Cargando JSON de Base de Datos ---")
# Datos zona - Base de datos (Nombres de zona)
print("--- Json Zona ---")
df_datos_zona_json = spark.read.json(hdfs_path_json)
df_datos_zona_json.show(5, truncate=False)


print("--- 3. Cargando CSVs de Ventas (Separador ',') ---")

# Valores Producto
print("--- CSV Valores Producto ---")
df_valores_producto = spark.read.csv(hdfs_path_val_prod, header=True, inferSchema=True, sep=",")
df_valores_producto.show(5, truncate=False)

# Unidades Producto
print("--- CSV Unidades Producto ---")
df_unidades_producto = spark.read.csv(hdfs_path_uni_prod, header=True, inferSchema=True, sep=",")
df_unidades_producto.show(5, truncate=False)

# Valores Familia
print("--- CSV Valores Familia ---")
df_valores_familia = spark.read.csv(hdfs_path_val_fam, header=True, inferSchema=True, sep=",")
df_valores_familia.show(5, truncate=False)

# Unidades Familia
print("--- CSV Unidades Familia ---")
df_unidades_familia = spark.read.csv(hdfs_path_uni_fam, header=True, inferSchema=True, sep=",")
df_unidades_familia.show(5, truncate=False)

print("--- ✅ Carga completada. Los 6 DataFrames estan listos. ---")

--- 1. Cargando Archivos Maestros (Separador ';') ---
--- CSV Maestra ---


+-----------+------------------+-------------------------------------------------+
|Producto   |Numero de articulo|Descripcion                                      |
+-----------+------------------+-------------------------------------------------+
|LAGRICEL PF|41582             |LAGRICEL OFTENO LIBRE DE CONSERVADORES (PF) 10 ML|
|ELIPTIC PF |41561             |ELIPTIC OFTENO 5ML PF PERU                       |
|LAGRICEL   |40515             |LAGRICEL OFTENO 0.5 ML                           |
|FLUMETOL NF|40513             |FLUMETOL NF OFTENO 5ML                           |
|TRAZIDEX O |40341             |TRAZIDEX OFTENO 5 ML.                            |
+-----------+------------------+-------------------------------------------------+
only showing top 5 rows

--- CSV Zona ---


+-----------+--------------------------------+-----------+-------+---+------+--------+
|Vendedor   |Nombre Cliente                  |Producto   |MES NUM|Mes|2025  |CANTIDAD|
+-----------+--------------------------------+-----------+-------+---+------+--------+
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A|AGGLAD     |1      |ENE|0     |0       |
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A|FLUMETOL NF|1      |ENE|0     |0       |
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A|GAAP       |1      |ENE|0     |0       |
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A|LAGRICEL   |1      |ENE|0     |0       |
|Pharma - N1|BENEL PEREZ,DENNY JAVIER        |FLUMETOL NF|1      |ENE|898.37|20      |
+-----------+--------------------------------+-----------+-------+---+------+--------+
only showing top 5 rows

--- 2. Cargando JSON de Base de Datos ---
--- Json Zona ---
+--------------------------+---------+------+---------+----------------------------------------------+-------+-----------+
|created_

25/11/18 02:19:56 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+----------+------------------+------------------+------------------+---------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+--------------------+------------+------------------+------------------+-------------------+-------------+------------------+------------------+---------+-----------------+------------------+------------------+-------------+--------------+------------------+------------------+----------+----------------+------------------+------------------+------------+----------------+------------------+------------------+------------+------------------+------------------+-------------+------------------+
|Zona       |Producto  |Enero _Venta      |Enero _T

### Exploración de datos

In [32]:
df_maestra.printSchema()
df_maestra.show(truncate=False)

root
 |-- Producto: string (nullable = true)
 |-- Numero de articulo: integer (nullable = true)
 |-- Descripcion: string (nullable = true)

+-----------+------------------+-------------------------------------------------+
|Producto   |Numero de articulo|Descripcion                                      |
+-----------+------------------+-------------------------------------------------+
|LAGRICEL PF|41582             |LAGRICEL OFTENO LIBRE DE CONSERVADORES (PF) 10 ML|
|ELIPTIC PF |41561             |ELIPTIC OFTENO 5ML PF PERU                       |
|LAGRICEL   |40515             |LAGRICEL OFTENO 0.5 ML                           |
|FLUMETOL NF|40513             |FLUMETOL NF OFTENO 5ML                           |
|TRAZIDEX O |40341             |TRAZIDEX OFTENO 5 ML.                            |
|TRAZIDEX U |40342             |TRAZIDEX UNGENA 3.5 G                            |
|SOPHIPREN  |40338             |SOPHIPREN OFTENO 5 ML                            |
|GAAP       |40498            

In [31]:
df_zona.printSchema()
df_zona.show(truncate=False)

root
 |-- Vendedor: string (nullable = true)
 |-- Nombre Cliente: string (nullable = true)
 |-- Producto: string (nullable = true)
 |-- MES NUM: integer (nullable = true)
 |-- Mes: string (nullable = true)
 |-- 2025: string (nullable = true)
 |-- CANTIDAD: string (nullable = true)

+-----------+-----------------------------------------------------------------+-----------+-------+---+-------+--------+
|Vendedor   |Nombre Cliente                                                   |Producto   |MES NUM|Mes|2025   |CANTIDAD|
+-----------+-----------------------------------------------------------------+-----------+-------+---+-------+--------+
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A                                 |AGGLAD     |1      |ENE|0      |0       |
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A                                 |FLUMETOL NF|1      |ENE|0      |0       |
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A                                 |GAAP       |1      |ENE|0      |0

In [30]:
df_datos_zona_json.printSchema()
df_datos_zona_json.show(truncate=False)

root
 |-- created_at: string (nullable = true)
 |-- is_active: boolean (nullable = true)
 |-- region: string (nullable = true)
 |-- zone_code: string (nullable = true)
 |-- zone_description: string (nullable = true)
 |-- zone_id: long (nullable = true)
 |-- zone_name: string (nullable = true)

+--------------------------+---------+--------+---------+----------------------------------------------+-------+-----------+
|created_at                |is_active|region  |zone_code|zone_description                              |zone_id|zone_name  |
+--------------------------+---------+--------+---------+----------------------------------------------+-------+-----------+
|2025-09-29 21:30:27.300355|true     |Centro  |Z1       |Zona de Lima Metropolitana                    |1      |Lima       |
|2025-09-29 21:30:27.300355|true     |Centro  |Z2       |Zona de Callao y Provincia Constitucional     |2      |Callao     |
|2025-09-29 21:30:27.300355|true     |Norte   |Z3       |Zona de Norte Chico - H

In [33]:
df_valores_producto.printSchema()
df_valores_producto.show(truncate=False)

root
 |-- Zona: string (nullable = true)
 |-- Producto: string (nullable = true)
 |-- Enero _Venta : double (nullable = true)
 |-- Enero _TGT : double (nullable = true)
 |-- Enero _PY 24: double (nullable = true)
 |-- Enero _% : integer (nullable = true)
 |-- Febrero _Venta : double (nullable = true)
 |-- Febrero _TGT : double (nullable = true)
 |-- Febrero _PY 24: double (nullable = true)
 |-- Febrero _% : double (nullable = true)
 |-- Marzo _Venta : double (nullable = true)
 |-- Marzo _TGT : double (nullable = true)
 |-- Marzo _PY 24: double (nullable = true)
 |-- Marzo _% : double (nullable = true)
 |-- Abril _Venta : double (nullable = true)
 |-- Abril _TGT : double (nullable = true)
 |-- Abril _PY 24: double (nullable = true)
 |-- Abril _% : double (nullable = true)
 |-- Mayo _Venta : double (nullable = true)
 |-- Mayo _TGT : double (nullable = true)
 |-- Mayo _PY 24: double (nullable = true)
 |-- Mayo _% : double (nullable = true)
 |-- Junio_Venta : double (nullable = true)
 |-- 

In [34]:
df_unidades_producto.printSchema()
df_unidades_producto.show(truncate=False)

root
 |-- Zona: string (nullable = true)
 |-- Producto: string (nullable = true)
 |-- Enero _Venta : integer (nullable = true)
 |-- Enero _TGT : integer (nullable = true)
 |-- Enero _PY 24: integer (nullable = true)
 |-- Enero _% : integer (nullable = true)
 |-- Febrero _Venta : integer (nullable = true)
 |-- Febrero _TGT : double (nullable = true)
 |-- Febrero _PY 24: integer (nullable = true)
 |-- Febrero _% : double (nullable = true)
 |-- Marzo _Venta : integer (nullable = true)
 |-- Marzo _TGT : double (nullable = true)
 |-- Marzo _PY 24: integer (nullable = true)
 |-- Marzo _% : double (nullable = true)
 |-- Abril _Venta : integer (nullable = true)
 |-- Abril _TGT : double (nullable = true)
 |-- Abril _PY 24: integer (nullable = true)
 |-- Abril _% : double (nullable = true)
 |-- Mayo _Venta : integer (nullable = true)
 |-- Mayo _TGT : double (nullable = true)
 |-- Mayo _PY 24: integer (nullable = true)
 |-- Mayo _% : double (nullable = true)
 |-- Junio_Venta : integer (nullable =

In [35]:
df_valores_familia.printSchema()
df_valores_familia.show(truncate=False)

root
 |-- Zona: string (nullable = true)
 |-- Producto: string (nullable = true)
 |-- Enero _Venta : double (nullable = true)
 |-- Enero _TGT : double (nullable = true)
 |-- Enero _PY 24: double (nullable = true)
 |-- Enero _% : integer (nullable = true)
 |-- Febrero _Venta : double (nullable = true)
 |-- Febrero _TGT : double (nullable = true)
 |-- Febrero _PY 24: double (nullable = true)
 |-- Febrero _% : double (nullable = true)
 |-- Marzo _Venta : double (nullable = true)
 |-- Marzo _TGT : double (nullable = true)
 |-- Marzo _PY 24: double (nullable = true)
 |-- Marzo _% : double (nullable = true)
 |-- Abril _Venta : double (nullable = true)
 |-- Abril _TGT : double (nullable = true)
 |-- Abril _PY 24: double (nullable = true)
 |-- Abril _% : double (nullable = true)
 |-- Mayo _Venta : double (nullable = true)
 |-- Mayo _TGT : double (nullable = true)
 |-- Mayo _PY 24: double (nullable = true)
 |-- Mayo _% : double (nullable = true)
 |-- Junio_Venta : double (nullable = true)
 |-- 

In [36]:
df_unidades_familia.printSchema()
df_unidades_familia.show(truncate=False)

root
 |-- Zona: string (nullable = true)
 |-- Producto: string (nullable = true)
 |-- Enero _Venta : integer (nullable = true)
 |-- Enero _TGT : integer (nullable = true)
 |-- Enero _PY 24: integer (nullable = true)
 |-- Enero _% : integer (nullable = true)
 |-- Febrero _Venta : integer (nullable = true)
 |-- Febrero _TGT : double (nullable = true)
 |-- Febrero _PY 24: integer (nullable = true)
 |-- Febrero _% : double (nullable = true)
 |-- Marzo _Venta : integer (nullable = true)
 |-- Marzo _TGT : double (nullable = true)
 |-- Marzo _PY 24: integer (nullable = true)
 |-- Marzo _% : double (nullable = true)
 |-- Abril _Venta : integer (nullable = true)
 |-- Abril _TGT : double (nullable = true)
 |-- Abril _PY 24: integer (nullable = true)
 |-- Abril _% : double (nullable = true)
 |-- Mayo _Venta : integer (nullable = true)
 |-- Mayo _TGT : double (nullable = true)
 |-- Mayo _PY 24: integer (nullable = true)
 |-- Mayo _% : double (nullable = true)
 |-- Junio_Venta : integer (nullable =

In [ ]:
from pyspark.sql.functions import col

# Quitar la fila "Producto"
df_familia_clean = df_familia.filter(df_familia["Producto_Familia"] != "Producto")
df_producto_clean = df_producto.filter(df_producto["Producto_Familia"] != "Producto")

# Aplicar unpivot corregido (con pivot final)
df_familia_wide = unpivot_to_wide(df_familia_clean)
df_producto_wide = unpivot_to_wide(df_producto_clean)

# Convertir columnas numéricas a double
num_cols = ["Venta", "TGT", "PY24", "pct"]

for c in num_cols:
    df_familia_wide = df_familia_wide.withColumn(c, col(c).cast("double"))
    df_producto_wide = df_producto_wide.withColumn(c, col(c).cast("double"))

# Mostrar resultados
print("=== Familia ===")
df_familia_wide.show(20, truncate=False)

print("=== Producto ===")
df_producto_wide.show(20, truncate=False)


=== Familia ===
+----------------+----------+---------+-----------+---------+-----------+
|Producto_Familia|Mes       |PY24     |TGT        |Venta    |pct        |
+----------------+----------+---------+-----------+---------+-----------+
|AGGLAD          |Abril     |24635.46 |120891.2407|91519.9  |0.757043269|
|AGGLAD          |Agosto    |25769.42 |2560.845455|0.0      |0.0        |
|AGGLAD          |Diciembre |71802.0  |2681.545455|0.0      |0.0        |
|AGGLAD          |Enero     |22534.35 |110094.81  |110094.81|1.0        |
|AGGLAD          |Febrero   |31785.06 |114546.1364|110784.62|0.96716156 |
|AGGLAD          |Julio     |29784.66 |133581.4492|18240.65 |0.136550772|
|AGGLAD          |Junio     |20271.95 |120891.2407|92445.3  |0.764698083|
|AGGLAD          |Marzo     |31785.06 |114546.1364|81102.53 |0.708033745|
|AGGLAD          |Mayo      |26193.67 |120891.2407|80999.82 |0.670022241|
|AGGLAD          |Noviembre |52414.29 |2681.545455|0.0      |0.0        |
|AGGLAD          |Octu

In [ ]:
from pyspark.sql.functions import trim, upper

# Normalizar columna Mes en ambos dataframes
df_familia_wide = df_familia_wide.withColumn("Mes", trim(upper(df_familia_wide["Mes"])))
df_producto_wide = df_producto_wide.withColumn("Mes", trim(upper(df_producto_wide["Mes"])))

In [ ]:
from pyspark.sql.functions import when, col

# Lista de métricas a convertir
metricas = ["Venta", "TGT", "PY24", "pct"]

# Familia
for m in metricas:
    df_familia_wide = df_familia_wide.withColumn(
        m,
        when(col(m).isNull(), 0.0).otherwise(col(m).cast("double"))
    )

# Producto
for m in metricas:
    df_producto_wide = df_producto_wide.withColumn(
        m,
        when(col(m).isNull(), 0.0).otherwise(col(m).cast("double"))
    )

In [ ]:
from pyspark.sql.functions import lit

# Agregar columna de nivel
df_familia_wide = df_familia_wide.withColumn("Nivel", lit("Familia"))
df_producto_wide = df_producto_wide.withColumn("Nivel", lit("Producto"))

# Unir ambos datasets
df_total = (
    df_familia_wide
    .unionByName(df_producto_wide, allowMissingColumns=True)
)

In [ ]:
df_total.show(100, truncate=False)

+----------------+----------+----------+-----------+---------+-----------+-------+
|Producto_Familia|Mes       |PY24      |TGT        |Venta    |pct        |Nivel  |
+----------------+----------+----------+-----------+---------+-----------+-------+
|AGGLAD          |ABRIL     |24635.46  |120891.2407|91519.9  |0.757043269|Familia|
|AGGLAD          |AGOSTO    |25769.42  |2560.845455|0.0      |0.0        |Familia|
|AGGLAD          |DICIEMBRE |71802.0   |2681.545455|0.0      |0.0        |Familia|
|AGGLAD          |ENERO     |22534.35  |110094.81  |110094.81|1.0        |Familia|
|AGGLAD          |FEBRERO   |31785.06  |114546.1364|110784.62|0.96716156 |Familia|
|AGGLAD          |JULIO     |29784.66  |133581.4492|18240.65 |0.136550772|Familia|
|AGGLAD          |JUNIO     |20271.95  |120891.2407|92445.3  |0.764698083|Familia|
|AGGLAD          |MARZO     |31785.06  |114546.1364|81102.53 |0.708033745|Familia|
|AGGLAD          |MAYO      |26193.67  |120891.2407|80999.82 |0.670022241|Familia|
|AGG

## Manejo de datos nulos


In [17]:
from pyspark.sql.functions import col

# Contar nulos en df_maestra
df_maestra.select([
    (col(c).isNull().cast("int")).alias(c) for c in df_maestra.columns
]).groupBy().sum().show()

# Contar nulos en df_zona
df_zona.select([
    (col(c).isNull().cast("int")).alias(c) for c in df_zona.columns
]).groupBy().sum().show()

+-------------+-----------------------+----------------+
|sum(Producto)|sum(Numero de articulo)|sum(Descripcion)|
+-------------+-----------------------+----------------+
|            0|                      0|               0|
+-------------+-----------------------+----------------+

+-------------+-------------------+-------------+------------+--------+---------+-------------+
|sum(Vendedor)|sum(Nombre Cliente)|sum(Producto)|sum(MES NUM)|sum(Mes)|sum(2025)|sum(CANTIDAD)|
+-------------+-------------------+-------------+------------+--------+---------+-------------+
|            0|                  0|            0|           0|       0|        0|            0|
+-------------+-------------------+-------------+------------+--------+---------+-------------+



In [18]:
# Eliminar filas con valores nulos en maestra
df_maestra_clean = df_maestra.na.drop()

# Eliminar filas con valores nulos en zona
df_zona_clean = df_zona.na.drop()


In [19]:
# Rellenar con texto en columnas de tipo string
df_maestra_filled = df_maestra.na.fill("Desconocido")

# Rellenar con 0 en columnas numéricas de df_zona
df_zona_filled = df_zona.na.fill(0)


## Correción de tipo de datos


In [20]:
df_maestra = df_maestra.withColumn("Numero de articulo", col("Numero de articulo").cast("int"))

In [21]:
df_zona = df_zona.withColumn("2025", col("2025").cast("double")) 

In [22]:
# Filtrado de datos irrelevantes

In [23]:
# Filtrar productos sin descripción
df_maestra = df_maestra.filter(col("Descripcion").isNotNull())

# Filtrar zonas con ventas registradas en cero
df_zona = df_zona.filter(col("2025") > 0)


## Union


In [24]:
df_zona = df_zona.filter(col("CANTIDAD") >= 0)
df_zona = df_zona.filter(col("2025") >= 0)

In [25]:
from pyspark.sql.functions import col

df_final = df_maestra.join(
    df_zona,
    df_maestra["Producto"] == df_zona["Producto"],
    how="inner"
).drop(df_zona["Producto"])

In [26]:
df_final.show()

+-----------+------------------+--------------------+-----------+--------------------+-------+---+-------+--------+
|   Producto|Numero de articulo|         Descripcion|   Vendedor|      Nombre Cliente|MES NUM|Mes|   2025|CANTIDAD|
+-----------+------------------+--------------------+-----------+--------------------+-------+---+-------+--------+
|FLUMETOL NF|             40513|FLUMETOL NF OFTEN...|Pharma - N1|BENEL PEREZ,DENNY...|      1|ENE| 898.37|      20|
| TRAZIDEX U|             40342|TRAZIDEX UNGENA 3...|Pharma - N1|BENEL PEREZ,DENNY...|      1|ENE|1028.61|      20|
|FLUMETOL NF|             40513|FLUMETOL NF OFTEN...|Pharma - N1|BM CLINICA DE OJO...|      1|ENE| 612.53|      15|
|       GAAP|             40498|    GAAP OFTENO 3 ML|Pharma - N1|BM CLINICA DE OJO...|      1|ENE|1925.85|      25|
| TRAZIDEX O|             40341|TRAZIDEX OFTENO 5...|Pharma - N1|BM CLINICA DE OJO...|      1|ENE| 250.05|       6|
| TRAZIDEX U|             40342|TRAZIDEX UNGENA 3...|Pharma - N1|BM CLIN

## Feature engeeniring

In [27]:
from pyspark.sql.functions import col, count, avg, sum, max, min, when, datediff, current_date


In [28]:
df_final = df_final.withColumn("Precio_Unitario", 
                               when(col("CANTIDAD") > 0, col("2025") / col("CANTIDAD")).otherwise(0))

In [29]:
df_final = df_final.withColumn("Precio_Unitario", 
                               when(col("CANTIDAD") > 0, col("2025") / col("CANTIDAD")).otherwise(0))

In [30]:
# 2. Gasto total por cliente
gasto_cliente = df_final.groupBy("Nombre Cliente").agg(sum("2025").alias("Gasto_Total"))

In [31]:
# 3. Cantidad total por cliente
cantidad_cliente = df_final.groupBy("Nombre Cliente").agg(sum("CANTIDAD").alias("Cantidad_Total"))

In [32]:
# 4. Promedio de gasto y cantidad por cliente
promedios_cliente = df_final.groupBy("Nombre Cliente").agg(
    avg("2025").alias("Gasto_Promedio"),
    avg("CANTIDAD").alias("Cantidad_Promedio")
)

# 5. Frecuencia de compra por cliente (número de transacciones)
frecuencia_cliente = df_final.groupBy("Nombre Cliente").agg(
    count("Producto").alias("Frecuencia_Compra")
)

# 6. Última compra (para calcular recencia)
ultima_compra = df_final.groupBy("Nombre Cliente").agg(
    max("MES NUM").alias("Ultimo_Mes")
)

In [33]:
# Combinar todas las features
features = gasto_cliente \
    .join(cantidad_cliente, "Nombre Cliente") \
    .join(promedios_cliente, "Nombre Cliente") \
    .join(frecuencia_cliente, "Nombre Cliente") \
    .join(ultima_compra, "Nombre Cliente")

# Mostrar resultado
features.show(10, truncate=False)

+-----------------------------------------+------------------+--------------+------------------+------------------+-----------------+----------+
|Nombre Cliente                           |Gasto_Total       |Cantidad_Total|Gasto_Promedio    |Cantidad_Promedio |Frecuencia_Compra|Ultimo_Mes|
+-----------------------------------------+------------------+--------------+------------------+------------------+-----------------+----------+
|CLINICA SAN FELIPE S A                   |21985.35          |700.0         |1099.2675         |35.0              |20               |7         |
|GOOD VISION E.I.R.L.                     |572.59            |12.0          |572.59            |12.0              |1                |3         |
|FONDO DE EMPLEADOS DEL BANCO DE LA NACION|8872.98           |229.0         |2957.66           |76.33333333333333 |3                |3         |
|CONSORCIO QORITAQQUE B Y T S.R.L.        |1427.8200000000002|29.0          |475.94000000000005|9.666666666666666 |3              

In [34]:
# Agregar df_total por producto (suma o promedio de todos los meses)
df_total_agg = df_total.groupBy("Producto_Familia", "Nivel").agg(
    sum("Venta").alias("Venta_Total_Anual"),
    sum("TGT").alias("Target_Total_Anual"),
    sum("PY24").alias("PY24_Total_Anual"),
    avg("pct").alias("pct_Promedio")
)

df_completo = df_final.join(
    df_total_agg,
    df_final["Producto"] == df_total_agg["Producto_Familia"],
    how="left"
)

In [35]:
df_completo.show()

+-----------+------------------+--------------------+-----------+--------------------+-------+---+-------+--------+------------------+----------------+--------+------------------+------------------+------------------+--------------------+
|   Producto|Numero de articulo|         Descripcion|   Vendedor|      Nombre Cliente|MES NUM|Mes|   2025|CANTIDAD|   Precio_Unitario|Producto_Familia|   Nivel| Venta_Total_Anual|Target_Total_Anual|  PY24_Total_Anual|        pct_Promedio|
+-----------+------------------+--------------------+-----------+--------------------+-------+---+-------+--------+------------------+----------------+--------+------------------+------------------+------------------+--------------------+
|FLUMETOL NF|             40513|FLUMETOL NF OFTEN...|Pharma - N1|BENEL PEREZ,DENNY...|      1|ENE| 898.37|      20|           44.9185|     FLUMETOL NF|Producto|         1621334.0|     1593594.73592|1884900.4700000002| 0.10436897615384616|
|FLUMETOL NF|             40513|FLUMETOL NF 

In [36]:
# Contar registros antes y después
print(f"Registros en df_final: {df_final.count()}")
print(f"Registros en df_completo: {df_completo.count()}")

# Deberían ser iguales si el join está bien hecho

Registros en df_final: 3840
Registros en df_completo: 5556


In [37]:
# Ruta de salida en HDFS
output_path = "hdfs://namenode:9000/user/nifi/processed/farmaceuticas/ventas_completo.csv"

# Guardar en CSV con encabezados y separador ;
df_completo.coalesce(1).write \
    .mode("overwrite") \
    .option("header", True) \
    .option("delimiter", ";") \
    .csv(output_path)